In [1]:
!pip install pandas scikit-learn joblib

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Anomaly detection function
def detect_anomalies(transactions):
    if transactions.empty:
        return False

    # Ensure 'amount' is numeric and 'timestamp' is datetime
    transactions['amount'] = pd.to_numeric(transactions['amount'], errors='coerce')
    transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])

    # Prepare the data for anomaly detection
    transactions['timestamp'] = transactions['timestamp'].astype(int) // 10**9  # Keep 'timestamp' as used in training

    # Load the pre-trained model
    model = joblib.load('anomaly_model.pkl')

    # Predict anomalies
    transactions['anomaly'] = model.predict(transactions[['amount', 'timestamp']])

    # Check for anomalies
    return transactions['anomaly'].value_counts().get(-1, 0) > 0


In [2]:
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_synthetic_data(num_records):
    data = []
    for _ in range(num_records):
        sender = f'user{random.randint(1, 10)}'
        recipient = f'user{random.randint(1, 10)}'
        amount = random.uniform(100, 10000)  # Random amounts between 100 and 10000
        timestamp = datetime.now() - timedelta(minutes=random.randint(1, 1440))  # Random time in last 24 hours
        data.append([sender, recipient, amount, timestamp.strftime("%Y-%m-%d %H:%M:%S")])
    return pd.DataFrame(data, columns=['sender', 'recipient', 'amount', 'timestamp'])

# Generate 50 synthetic transaction records
synthetic_data = generate_synthetic_data(50)
synthetic_data.to_csv('transactions.csv', mode='a', header=False, index=False)  # Append to the CSV


In [3]:
import pandas as pd

# Load your transaction data
transaction_file = 'transactions.csv'
new_training_data = pd.read_csv(transaction_file)

# Convert timestamp to a usable format (e.g., seconds)
new_training_data['timestamp'] = pd.to_datetime(new_training_data['timestamp'])
new_training_data['time'] = new_training_data['timestamp'].astype(int) // 10**9  # Convert to seconds

# Ensure that 'amount' is numeric
new_training_data['amount'] = pd.to_numeric(new_training_data['amount'], errors='coerce')


In [4]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Generate synthetic transactions for the user 'sowmya'
def generate_transactions():
    data = []
    current_balance = 10000  # Sowmya's balance
    sender = 'sowmya'
    
    for i in range(10):
        recipient = f'user{random.randint(1, 5)}'
        amount = random.uniform(500, 3000)  # Random amounts between 500 and 3000
        timestamp = datetime.now() - timedelta(minutes=random.randint(1, 30))
        current_balance -= amount
        data.append([sender, recipient, amount, timestamp.strftime("%Y-%m-%d %H:%M:%S")])
    
    return pd.DataFrame(data, columns=['sender', 'recipient', 'amount', 'timestamp'])

transactions_df = generate_transactions()
transactions_df.to_csv('transactions.csv', index=False)
print(transactions_df)


   sender recipient       amount            timestamp
0  sowmya     user5  1828.161106  2024-10-19 16:19:10
1  sowmya     user4   929.537064  2024-10-19 16:26:10
2  sowmya     user3  1254.617754  2024-10-19 16:27:10
3  sowmya     user2  2000.109460  2024-10-19 16:33:10
4  sowmya     user5  1902.119625  2024-10-19 16:15:10
5  sowmya     user3  1466.961642  2024-10-19 16:30:10
6  sowmya     user1  1721.083778  2024-10-19 16:29:10
7  sowmya     user2  1196.425798  2024-10-19 16:17:10
8  sowmya     user1  1981.100120  2024-10-19 16:12:10
9  sowmya     user3  2770.555813  2024-10-19 16:36:10


In [5]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Generate synthetic transactions for the user 'sowmya'
def generate_transactions():
    data = []
    current_balance = 10000  # Sowmya's balance
    sender = 'sowmya'
    
    for i in range(10):
        recipient = f'user{random.randint(1, 5)}'
        amount = random.uniform(500, 3000)  # Random amounts between 500 and 3000
        timestamp = datetime.now() - timedelta(minutes=random.randint(1, 30))
        current_balance -= amount
        data.append([sender, recipient, amount, timestamp.strftime("%Y-%m-%d %H:%M:%S")])
    
    return pd.DataFrame(data, columns=['sender', 'recipient', 'amount', 'timestamp'])

transactions_df = generate_transactions()
transactions_df.to_csv('transactions.csv', index=False)
print(transactions_df)


   sender recipient       amount            timestamp
0  sowmya     user5  2465.950086  2024-10-19 16:25:05
1  sowmya     user3   827.760248  2024-10-19 16:24:05
2  sowmya     user1  1759.332205  2024-10-19 16:20:05
3  sowmya     user3  2248.600975  2024-10-19 16:19:05
4  sowmya     user1  1245.746257  2024-10-19 16:34:05
5  sowmya     user4  2752.461914  2024-10-19 16:14:05
6  sowmya     user4  1768.068608  2024-10-19 16:37:05
7  sowmya     user1  2341.297248  2024-10-19 16:31:05
8  sowmya     user3  1449.017237  2024-10-19 16:22:05
9  sowmya     user1  1602.062611  2024-10-19 16:23:05


In [6]:
def detect_anomalies(transactions, user_balance):
    anomalies = []
    
    # Rule 1: Detect unusually large transactions (50% of balance)
    large_transaction_limit = user_balance * 0.5
    large_transactions = transactions[transactions['amount'] > large_transaction_limit]
    
    if not large_transactions.empty:
        anomalies.append(f"Unusually large transaction(s): {large_transactions[['recipient', 'amount']].to_dict()}")

    # Rule 2: Detect multiple transactions within a short period
    transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])
    transactions['time_diff'] = transactions['timestamp'].diff().dt.seconds / 60  # Time difference in minutes
    rapid_transactions = transactions[transactions['time_diff'] < 5]  # More than 1 transaction in under 5 minutes
    
    if not rapid_transactions.empty:
        anomalies.append(f"Multiple transactions within short period: {rapid_transactions[['recipient', 'amount']].to_dict()}")

    # Rule 3: New recipient detection
    known_recipients = {'user1', 'user2'}  # Define a set of known recipients
    new_recipients = transactions[~transactions['recipient'].isin(known_recipients)]
    
    if not new_recipients.empty:
        anomalies.append(f"Transaction(s) to unknown recipient(s): {new_recipients[['recipient', 'amount']].to_dict()}")
    
    return anomalies

# Detect anomalies in the user's transactions
user_balance = 10000  # Initial balance of Sowmya
anomalies_detected = detect_anomalies(transactions_df, user_balance)

if anomalies_detected:
    for anomaly in anomalies_detected:
        print(f"Alert: {anomaly}")


Alert: Multiple transactions within short period: {'recipient': {9: 'user1'}, 'amount': {9: 1602.062611426888}}
Alert: Transaction(s) to unknown recipient(s): {'recipient': {0: 'user5', 1: 'user3', 3: 'user3', 5: 'user4', 6: 'user4', 8: 'user3'}, 'amount': {0: 2465.95008636651, 1: 827.7602475627292, 3: 2248.6009745759507, 5: 2752.4619139752676, 6: 1768.0686076440331, 8: 1449.0172367618766}}


In [2]:
from twilio.rest import Client

# Your Twilio credentials
account_sid = 'AC914bdf37e96d992612a35e39ecc8c491'
auth_token = '6413e9cf42276d7b9bd9bad5048361d6'
client = Client(account_sid, auth_token)

def send_twilio_alert(anomalies):
    message_body = f"Anomaly detected in your account: {', '.join(anomalies)}"
    
    # Replace 'your_twilio_number' with your actual Twilio number and 'your_phone_number' with the user's number
    message = client.messages.create(
        body=message_body,
        from_='+13057833745',
        to='+91 9363275090'
    )
    return message.sid


In [3]:
# Function to log transaction
def log_transaction(sender, recipient, amount):
    transaction_data = {
        "sender": sender,
        "recipient": recipient,
        "amount": amount,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    try:
        df = pd.DataFrame([transaction_data])
        df.to_csv(transaction_file, mode='a', index=False, header=not pd.read_csv(transaction_file).shape[0])
    except FileNotFoundError:
        df.to_csv(transaction_file, mode='w', index=False, header=True)
    except pd.errors.ParserError:
        print("Error logging the transaction. CSV format issue.")
